## VeggedIn
## - Abhishek Nair
## - priyanshi Furiya
## Problem Statement 5 - DataBytes

### Setup

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d misrakahmed/vegetable-image-dataset

 98% 523M/534M [00:05<00:00, 122MB/s]
100% 534M/534M [00:05<00:00, 102MB/s]


In [ ]:
!pip install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import patoolib
patoolib.extract_archive("/content/vegetable-image-dataset.zip")
!rm /content/vegetable-image-dataset.zip

INFO patool: Extracting /content/vegetable-image-dataset.zip ...
INFO:patool:Extracting /content/vegetable-image-dataset.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_oa5agfcs -- /content/vegetable-image-dataset.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_oa5agfcs -- /content/vegetable-image-dataset.zip
INFO patool:     with input=''
INFO:patool:    with input=''


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os, shutil
from PIL import Image
from PIL import ImageEnhance
from skimage.io import imread
import random, pathlib, warnings, itertools, math, os, shutil
from tensorflow.keras import layers
warnings.filterwarnings('ignore')

### Data Loading

In [ ]:
dataset= "/content/Vegetable Images"

train_folder = os.path.join(dataset,"train")
test_folder = os.path.join(dataset,"validation")
validation_folder = os.path.join(dataset,"test")

### Model

In [ ]:
train_data_generator = ImageDataGenerator(
    rescale=1/255.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rotation_range=10
)

val_data_generator = ImageDataGenerator(
    rescale=1/255.,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_data_generator = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    train_folder,
    shuffle=True,
    class_mode='binary',
    batch_size=32
)

val_generator = val_data_generator.flow_from_directory(
    validation_folder,
    shuffle=False,
    class_mode='binary',
    batch_size=32
)

test_generator = test_data_generator.flow_from_directory(
    test_folder,
    shuffle=False,
    class_mode='binary',
    batch_size=32
)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [ ]:
train_class_indices = train_generator.class_indices
class_map = {v: k for k, v in train_class_indices.items()}
class_map

{0: 'Bean',
 1: 'Bitter_Gourd',
 2: 'Bottle_Gourd',
 3: 'Brinjal',
 4: 'Broccoli',
 5: 'Cabbage',
 6: 'Capsicum',
 7: 'Carrot',
 8: 'Cauliflower',
 9: 'Cucumber',
 10: 'Papaya',
 11: 'Potato',
 12: 'Pumpkin',
 13: 'Radish',
 14: 'Tomato'}

In [ ]:
pretrained = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=[256,256,3], include_top=False,
    classifier_activation='softmax',
)
pretrained.trainable = False
model = tf.keras.models.Sequential([
    pretrained,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(15, activation='softmax')
])

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 8, 8, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 2947663 (11.24 MB)
Trainable params: 68967

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor="val_loss",patience=5, restore_best_weights=True)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=epochs,
                    callbacks=[checkpoint_callback,early_stopping_callback])

Epoch 1/15
469/469 [==============================] - 320s 668ms/step - loss: 0.1177 - accuracy: 0.9675 - val_loss: 0.0274 - val_accuracy: 0.9910
Epoch 2/15
469/469 [==============================] - 274s 585ms/step - loss: 0.0304 - accuracy: 0.9912 - val_loss: 0.0423 - val_accuracy: 0.9877
Epoch 3/15
469/469 [==============================] - 272s 580ms/step - loss: 0.0204 - accuracy: 0.9941 - val_loss: 0.1042 - val_accuracy: 0.9730
Epoch 4/15
469/469 [==============================] - 273s 582ms/step - loss: 0.0217 - accuracy: 0.9942 - val_loss: 0.0328 - val_accuracy: 0.9920
Epoch 5/15
469/469 [==============================] - 311s 663ms/step - loss: 0.0154 - accuracy: 0.9954 - val_loss: 0.0138 - val_accuracy: 0.9947
Epoch 6/15
469/469 [==============================] - 278s 593ms/step - loss: 0.0110 - accuracy: 0.9965 - val_loss: 0.0093 - val_accuracy: 0.9970
Epoch 7/15
469/469 [==============================] - 275s 586ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.0231 -

In [ ]:
model = tf.keras.models.load_model('/content/final_model.h5')
# predicted_vegetable = generate_predictions("/content/batata.jpg")
# print(predicted_vegetable)

In [ ]:
def generate_predictions(image_path):
    test_img = image.load_img(image_path, target_size=(256, 256))
    test_img_arr = image.img_to_array(test_img)/255.0
    test_img_input = test_img_arr.reshape((1, test_img_arr.shape[0], test_img_arr.shape[1], test_img_arr.shape[2]))

    predicted_label = np.argmax(model.predict(test_img_input))
    predicted_vegetable = class_map[predicted_label]
    return predicted_vegetable

In [ ]:
generate_predictions("/content/batata.jpg")

1/1 [==============================] - 0s 22ms/step


'Potato'

### Quantity (Object Detection)

In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0

In [ ]:
from google.colab import userdata

In [ ]:
from roboflow import Roboflow
#replace (userdata.get('RoboFlowAPI') with oTFdgfpUXCaDBg8v2Is6 if not working
# rf = Roboflow(api_key=(userdata.get('RoboFlowAPI')))
rf = Roboflow(api_key="oTFdgfpUXCaDBg8v2Is6")
project = rf.workspace().project("test-veg")
ModelCount = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
def count_instances(predictions: dict) -> dict:
    classes = {}

    for bounding_box in predictions['predictions']:
        if bounding_box['class'] in classes:
            classes[bounding_box['class']] += 1
        else:
            classes[bounding_box['class']] = 1

    return classes

In [ ]:
prediction = ModelCount.predict("/content/images.jpeg", confidence=40, overlap=30).save("prediction.jpg")

In [ ]:
count_instances(prediction)

### Calculating average weight


In [ ]:
def calculate_weight_for_predicted_vegetable(
    predicted_vegetable, quantity_dict, csv_file_path
):
    df = pd.read_csv(csv_file_path, encoding="latin1")

    df["Product"] = df["Product"].str.lower()

    product_to_weight = dict(zip(df["Product"], df["Weight"]))

    predicted_vegetable_lower = predicted_vegetable.lower()

    if predicted_vegetable_lower not in quantity_dict:
        print("Error Predicting Vegetable: Predicted vegetable not found in quantity_dict.")
        return None

    if predicted_vegetable_lower in product_to_weight:
        predicted_quantity = quantity_dict.get(predicted_vegetable_lower, 0)
        total_weight = predicted_quantity * product_to_weight[predicted_vegetable_lower]
        return total_weight
    else:
        print(
            f"Warning: Predicted vegetable '{predicted_vegetable}' not found in the CSV file."
        )
        return None

### Integration

In [ ]:
def generate_predictions_with_count(image_path):
    predicted_vegetable = generate_predictions(image_path)
    prediction = ModelCount.predict(image_path, confidence=40, overlap=30).json()
    count = count_instances(prediction)
    return predicted_vegetable, count


In [ ]:
csv_path = "/content/WeightMapping.csv"

In [ ]:
import ipywidgets as widgets
from IPython.display import display

quantity_dict = {
    'bean': 0, 'bitter_gourd': 0, 'bottle_gourd': 0, 'brinjal': 0,
    'broccoli': 0, 'cabbage': 0, 'capsicum': 0, 'carrot': 0,
    'cauliflower': 0, 'cucumber': 0, 'papaya': 0, 'potato': 0,
    'pumpkin': 0, 'radish': 0, 'tomato': 0
}

list_input = widgets.Text(
    value='',
    placeholder='Enter the list of vegetables and quantities (e.g., bean:3,carrot:2,tomato:4)',
    description='Vegetable List:'
)

submit_button = widgets.Button(description='Submit')
display(widgets.HBox([list_input, submit_button]))
image_path_input = widgets.Text(
    value='',
    placeholder='Enter the path to the image',
    description='Image Path:'
)
output = widgets.Output()

def on_submit_click(b):
    global items
    items = None
    try:
        user_input = list_input.value
        items = [item.split(':') for item in user_input.split(',')]
        for vegetable, quantity in items:
            quantity_dict[vegetable.strip().lower()] += int(quantity)
        with output:
            output.clear_output()
            print(f"Updated Purchase List: {quantity_dict}")

        display(image_path_input)

    except ValueError:
        with output:
            output.clear_output()
            print("Error: Invalid input format. Please enter quantities as integers.")

submit_button.on_click(on_submit_click)

final_output = widgets.Output()
display(final_output)

def on_image_path_submit(b):
    global items, quantity_dict
    image_path = image_path_input.value
    predicted_vegetable, count_dict = generate_predictions_with_count(image_path)
    for vegetable, count in count_dict.items():
        vegetable_lower = vegetable.lower()
        if vegetable_lower in quantity_dict:
            subtracted_count = min(count, quantity_dict[vegetable_lower])
            quantity_dict[vegetable_lower] -= subtracted_count

    predicted_vegetable_weight = calculate_weight_for_predicted_vegetable(
        predicted_vegetable, count_dict, csv_path
    )

    with final_output:
        final_output.clear_output()
        if predicted_vegetable_weight is not None:
            print(f"Predicted Vegetable: {predicted_vegetable}")
            print(f"Updated Purchase List: {quantity_dict}")
            print(f"Total Weight: {predicted_vegetable_weight} grams")
        else:
            print("Error calculating weight.")


image_path_submit_button = widgets.Button(description='Submit Image Path')
image_path_submit_button.on_click(on_image_path_submit)
display(image_path_submit_button)

### FrontEnd

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=284873e85ff7e6880ce8dad214bb76b8d5f24284d73a01036323d0a675789b66
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_P6DBK2FAFASD7YJURYWZIDXK-PQARQF3OUZ6246MT")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
quantity_dict = {
    'bean': 0, 'bitter_gourd': 0, 'bottle_gourd': 0, 'brinjal': 0,
    'broccoli': 0, 'cabbage': 0, 'capsicum': 0, 'carrot': 0,
    'cauliflower': 0, 'cucumber': 0, 'papaya': 0, 'potato': 0,
    'pumpkin': 0, 'radish': 0, 'tomato': 0
}

@anvil.server.callable
def process_input(vegetable, quantity):
    global quantity_dict

    try:
        if vegetable.strip().lower() == 'reset':
            quantity_dict = {
                'bean': 0, 'bitter_gourd': 0, 'bottle_gourd': 0, 'brinjal': 0,
                'broccoli': 0, 'cabbage': 0, 'capsicum': 0, 'carrot': 0,
                'cauliflower': 0, 'cucumber': 0, 'papaya': 0, 'potato': 0,
                'pumpkin': 0, 'radish': 0, 'tomato': 0
            }
            return {"reset": True, "updated_vegetables": {}}
        else:
            vegetable_lower = vegetable.strip().lower()
            if vegetable_lower in quantity_dict:
                quantity_dict[vegetable_lower] += int(quantity)
                updated_vegetables = {vegetable_lower: quantity_dict[vegetable_lower]}
                return {"reset": False, "updated_vegetables": updated_vegetables}
            else:
                return {"error": f"Invalid vegetable: {vegetable}"}

    except ValueError:
        return {"error": "Invalid input format. Please enter quantities as integers."}


In [ ]:
@anvil.server.callable
def enter_image_path(image_path):
    global quantity_dict

    try:
        predicted_vegetable, count_dict = generate_predictions_with_count(image_path)
        for vegetable, count in count_dict.items():
            vegetable_lower = vegetable.lower()
            if vegetable_lower in quantity_dict:
                subtracted_count = min(count, quantity_dict[vegetable_lower])
                quantity_dict[vegetable_lower] -= subtracted_count
        predicted_vegetable_weight = calculate_weight_for_predicted_vegetable(
            predicted_vegetable, count_dict, csv_path
        )

        return {
            "predicted_vegetable": predicted_vegetable,
            "updated_purchase_list": quantity_dict,
            "total_weight": predicted_vegetable_weight
        }

    except ValueError:
        return {"error": "Error processing image path."}


In [ ]:
@anvil.server.callable
def get_shopping_list():
    global quantity_dict
    return quantity_dict


In [ ]:
anvil.server.wait_forever()

1/1 [==============================] - 0s 59ms/step
